### Instructions: Follow the instructions provided in each step, or in the output from a cell 
Step 1
* Make sure the python notebook and the pivot table Excel file are in the **same** folder on your computer.
* Enter the name of the pivot table in the next cell, then press the 'run' button above.
* The python notebook will give you a preview of what will be used in the analysis beneath the cell after you press the run button.  
  * Scan the row names to verify that they are the same as in the Excel sheet that you want ot use.  If they are, skip the following cells and proceed to Step 2.
* Otherwise, follow the instructions in the output.

In [1]:
pivot_table = '20411_Felber1-2_MHC-I_Haplotypes_23Mar18.xlsx' # enter name here

In [2]:
import pandas as pd
from pandas import DataFrame
import re
import time
import math

def parseExcelWithPandas(fName):
    eSheetData = ''
    dfs = {sheet_name: testFile.parse(sheet_name)
      for sheet_name in testFile.sheet_names}
    if not fName:
        sheetCount = -1
        for s in dfs:
            sheetCount += 1
            m = re.search('pivot', s)
            m2 = re.search('MiSeq', s)
            if m:
                # JRC Comment:
                # using pd.read_excel(pivot_table, sheet_name=excelSheetName)
                # is vulnerable to a bug whereby pandas will intermittently ignore parameters and grab the first sheet only, 
                # UNLESS the 0-indexed integer for the sheet is specified with the function x = pd.ExcelFile() followed by x.parse(int)
                eSheetDataInt = pd.ExcelFile(pivot_table)
                eSheetData = eSheetDataInt.parse(sheetCount)
            elif m2:
                eSheetDataInt = pd.ExcelFile(pivot_table)
                eSheetData = eSheetDataInt.parse(sheetCount)
            else:
                continue
    else:
        for s in dfs:
            m = re.search(fName, s) # case insensitive?
            if m:
                eSheetDataInt = pd.ExcelFile(pivot_table)
                eSheetData = eSheetDataInt.parse(sheetCount)
            else:
                continue
    return eSheetData

def findColumnIdxStartStop(pdDF):
    xCt = -1
    xStart = -1
    xStop = -1
    foundInitialMatch = False
    for x in excelSheetName.columns.values:
        xCt += 1
        if xCt == 0:
            continue
        else:
            if xCt < 10 and foundInitialMatch == False: # column idx will not be greater than 9
                m = re.search('named', str(x))
                if m:
                    continue
                else:
                    foundInitialMatch = True
                    xStart = xCt
            else: 
                m = re.search('named', str(x))
                if m:
                    xStop = xCt
                    break
    return (xStart, xStop)
def parsePandasDfRows(col1ListFromPdDf):
    headers = True
    mamuA_indices = []
    mamuB_indices = []
    skipIndices = []
    genotypeList = []
    for idx,val in enumerate(col1ListFromPdDf):
        if headers:
            m = re.search('Comment', str(val))
            mA = re.search('Mamu-A', str(val))
            mB = re.search('Mamu-B', str(val))
            if m:
                headers = False
                skipIndices.append(idx)
                continue
            elif mA:
                mamuA_indices.append(idx)
            elif mB:
                mamuB_indices.append(idx)
            else:
                skipIndices.append(idx)
                continue
        else:
            m = re.search('Alleles', str(val))
            if m:
                skipIndices.append(idx)
            else:
                genotypeList.append(val)
                continue
    return (skipIndices, mamuA_indices, mamuB_indices, genotypeList)
def dataToPandasOneHot(d, pdDf, idxList):
    if pdDf is None:
        pdDf = pd.DataFrame.from_dict(d, orient='index').transpose()
        pdDf.index = idxList
        pdDf.index.name = 'genotype'
    else:
        alleleDFnew = pd.DataFrame.from_dict(d, orient='index').transpose()
        alleleDFnew.index = idxList
        alleleDFnew.index.name = 'genotype'
        alleleDFres = pd.concat([pdDf, alleleDFnew], axis=1, join_axes=[pdDf.index])
        pdDf = alleleDFres
    return pdDf
def parseIdxForMamuAMamuB(gList):
    mamuA_nameIdxListCol1 = []
    mamuB_nameIdxListCol1 = []
    for idx, n in enumerate(gList):
        m_MamuA = re.search('Mamu_A', str(n))
        m_MamuB = re.search('Mamu_B', str(n))
        if m_MamuA:
            filterMamu = re.search('Mamu_AG', str(n))
            if filterMamu:
                continue
            else:
                mamuA_nameIdxListCol1.append(idx)
        elif m_MamuB:
            mamuB_nameIdxListCol1.append(idx)
        else:
            continue
    return (mamuA_nameIdxListCol1, mamuB_nameIdxListCol1)

def parseGenotypeList(gList):
    r = []
    for g in gList:
        gString = g.split('_')
        gStringAsList = gString[1:]
        gStringAsString = '_'.join(gStringAsList)
        r.append(gStringAsString)
    return r

def parseGenotypeTuple(gList):
    r = []
    

def scanAndUpdateGenotypeList(l1_parsed,l2_training):
    # scan for entry.  if entry == True AND count == 1, continue ; 
    # else: grep for entry in l2_training, find first AND closest match
    # # If find first and closest match true:
    # # # re-scan l2_training using matched entry.  
    # # # IFF count of matches == 1 in l2_training:
    # # # # return matchName modified from X to be Y, where X is the previous name in l1_parsed, Y is the new name in l1_parsed
    # # # # else: return 'No match possible'
    # # else: return 'No match possible'
    l1_parsed_res = []
    for i in l1_parsed:
        didFindMatch = False
        if (l2_training.count(i) == 1):
            l1_parsed_res.append((1, i))
            continue
        else:
            iString = i.split('g')
            iStringRgx = iString[0]
            for itm in l2_training:
                m1 = re.search(iStringRgx, itm):
                    if m1:
                        mString = m1.group(0)
                        if (l2_training.count(mString) == 1):
                            l1_parsed_res.append((2, mString))
                            didFindMatch = True
                            print('WARNING: replacing original item ' + str(i) + ' with modified match from training set: ' + str(mString))
                            break # this is important: it stops at the first match
                    else:
                        continue
            if not didFindMatch:
                print('WARNING! Unable to match item ' + str(i))
                print('Item ' + str(i) + ' was deleted from dataset.\nThis behavior will be modified in a future release.')
                l1_parsed_res.append((0, i))
            else:
                continue
    return l1_parsed_res
    
testFile = pd.ExcelFile(pivot_table)
readyToProceed = False
fName_none = ''
dataAsPandas = ''
excelSheetName = parseExcelWithPandas(fName_none)
processedPivotTable = False
if pivot_table:
    if excelSheetName.empty:
        print('The Excel sheet was found, but there was an error reading the Excel file.')
        print('Please do one of the following:\n\nProceed to the next cell and attempt to enter the sheet_name value,\n\nor\n\nExport the file as a csv, start from the beginning of the python notebook,\nchange the file name, and rerun all cells.\nBe sure to specify the file type as csv in the cell below when you run it.')
    else:
        print('Please click the next cell, and press Run.')
        print('Here is a preview of the data: \n\n##StartPreview:\n')
        pdHeadersRowsCol1 = list(excelSheetName.iloc[1:10:,0])
        for h in pdHeadersRowsCol1:
            m = re.search('Animal ID', h)
            if m:
                processedPivotTable = True
            print(h + '\n')
        print('##EndPreview\n\nData Type is ')
        if processedPivotTable:
            print('Processed Pivot Table')
        else:
            print('raw pivot table')
        readyToProceed = True
else:
    print('No pivot table was found. Please do the following: \n1) check the filename and rerun the previous cell\n2) if you have already rerun the previous cell and are seeing this message again, \nproceed to the next cell and enter information for at least one of the following: ')
    print('\tfile_type: enter \'csv\' or \'excel\', depending on the file.')
    print('\tsheet_name: enter the sheet name for the excel spreadsheet or csv file')
    print('Then run the next two cells.')

Please click the next cell, and press Run.
Here is a preview of the data: 

##StartPreview:

Animal ID

# Reads Evaluated

# Reads Identified

% Unknown

Mamu-A Haplotype 1

Mamu-A Haplotype 2

Mamu-B Haplotype 1

Mamu-B Haplotype 2

Comments

##EndPreview

Data Type is 
Processed Pivot Table


In [3]:
file_type = '' # must be left blank, 'csv' for a csv file, or 'excel' for an excel file type
sheet_name = '' # must have the excel sheet name with the pivot table, or be blank

In [18]:
# readyToProceed = False
skipBool = False
if readyToProceed:
    print('It looks like you are running this cell after the data has already been set up,\nso this cell will be skipped.\nIf you are certain that you want to rerun this cell anyway, uncomment the first line,\nand click the run button again.')
else:
    if sheet_name:
        excelSheetName = parseExcelWithPandas(sheet_name)
        if excelSheetName.empty:
            skipBool = True
            print('Unable to find Excel sheet named ' + str(sheet_name))
        else:
            print('Everything looks good!')
            print('Here is a preview of the data: \n')
            pdHeadersRowsCol1 = list(excelSheetName.iloc[1:10:,0])
            for h in pdHeadersRowsCol1:
                print(h + '\n')
            readyToProceed = True
    elif file_type:
        if file_type == 'csv':
            csvFileConv = pivot_table.split('.')
            csvFileConvString = '.'.join(csvFileConv[:-1])
            csvFileConvName = csvFileConvString + '.csv'
            try:
                dataAsPandas = pd.read_csv(csvFileConvName)
            except FileNotFoundError:
                print('csv file ' + csvFileConvName + ' not found.\nPlease verify it is named correctly and that it exists.')
                skipBool = True
            except UnicodeDecodeError:
                print('please make sure that the file ' + pivot_table + '\nhas been converted correctly to csv format. ')
                skipBool = True
                dfs = pd.read_excel(file_name, sheet_name=None)
        elif file_type == 'excel':
            excelSheetName = parseExcelWithPandas(sheet_name)
            if excelSheetName.empty:
                skipBool = True
                print('Unable to parse Excel sheet')
            else:
                print('Everything looks good!')
                print('Here is a preview of the data: \n')
                pdHeadersRowsCol1 = list(excelSheetName.iloc[1:10:,0])
                for h in pdHeadersRowsCol1:
                    print(h + '\n')
                readyToProceed = True
        else:
            print('File Type not recognized.')
            skipBool = True
    else:
        print('It looks like you have not entered any additional data.\nThe workflow cannot proceed unless you specify the file type or if you specify the sheet name, or both.')
if skipBool:
    print('Please re-enter data in the cell above, then rerun that cell and this one.')

It looks like you are running this cell after the data has already been set up,
so this cell will be skipped.
If you are certain that you want to rerun this cell anyway, uncomment the first line,
and click the run button again.


Step 2
* This step will reformat the Excel data into a format that can be used by the Machine Learning Classifier.
* Click the next cell, and then click the Run button at the top.
* If you do not see any error messages, and you see the output 'Everything looks good!', then proceed to Step 3.  Otherwise, contact John for assistance with Step 2
  * Note from John: In the next Beta build (1.2) this step will be modified to not require intervention from me if something goes wrong.
* If you see a warning message, it is still usually ok to proceed, but you should make a note of the warning.


In [5]:
v = findColumnIdxStartStop(excelSheetName)
if v[0] == -1:
    print('Error!  check dataframe!')
col1List = list(excelSheetName['Sample Sheet #'])
skipRows, mamuA_rows, mamuB_rows, genotypeList_unparsed = parsePandasDfRows(col1List)
genotypeList_parsed = parseGenotypeList(genotypeList_unparsed)
genotypeListTuple = scanAndUpdateGenotypeList(genotypeList_parsed, genotypeList_training)
skipMamuRows = mamuA_rows + mamuB_rows + skipRows
skipMamuRows.sort()
# final step: parse through genotypeListTuple, and remove any rows that correspond to (0, name)
genotypeListFiltered = list(filter(lambda x: x[0] != 0, genotypeListTuple))
genotypeListInts, genotypeList = zip(*genotypeListFiltered) # zip iterable, unpacks tuple
parsedMamuIndices = parseIdxForMamuAMamuB(genotypeList)
parsedMamuAIndices = parsedMamuIndices[0]
parsedMamuBIndices = parsedMamuIndices[1]

print(genotypeList)
rStart = v[0]
rStop = v[1] + 1 
mamuA_alleleList = []
mamuB_alleleList = []
alleleDF_MamuA = None
alleleDF_MamuB = None
for x in range(rStart, rStop):
    mamu_genotypes_oneHot = []
    dfValue = excelSheetName.iloc[:,x] # verify syntax for rows and columns, and index not lookup
    dfValue_MamuA_1 = dfValue.iloc[int(mamuA_rows[0])]
    dfValue_MamuA_2 = dfValue.iloc[int(mamuA_rows[1])]
    dfValue_MamuB_1 = dfValue.iloc[int(mamuB_rows[0])]
    dfValue_MamuB_2 = dfValue.iloc[int(mamuB_rows[1])]
    pdDict_MamuA = dict()
    pdDict_MamuB = dict()
    for idx, row in dfValue.iteritems():
        if idx not in skipMamuRows:
            try:
                if math.isnan(row):
                    mamu_genotypes_oneHot.append(0)
                    continue
                else:
                    mamu_genotypes_oneHot.append(1)
            except TypeError:
                mamu_genotypes_oneHot.append(1)
    pdDictKey_MamuA = str(dfValue_MamuA_1) + '-' + str(dfValue_MamuA_2)
    pdDictKey_MamuB = str(dfValue_MamuB_1) + '-' + str(dfValue_MamuB_2)
    pdDict_MamuA[pdDictKey_MamuA] = mamu_genotypes_oneHot
    pdDict_MamuB[pdDictKey_MamuB] = mamu_genotypes_oneHot
    alleleDF_MamuA = dataToPandasOneHot(pdDict_MamuA, alleleDF_MamuA, genotypeList)
    alleleDF_MamuB = dataToPandasOneHot(pdDict_MamuB, alleleDF_MamuB, genotypeList)
alleleDF_MamuA_parsed = alleleDF_MamuA.iloc[parsedMamuAIndices,:]
alleleDF_MamuB_parsed = alleleDF_MamuB.iloc[parsedMamuBIndices,:]
print(alleleDF_MamuA_parsed)
if not alleleDF_MamuA_parsed.empty and not alleleDF_MamuB_parsed.empty:
    print('Everything looks good!')

['01_Mamu_A1_001g2', '01_Mamu_A1_002_01', '01_Mamu_A1_002_w_02', '01_Mamu_A1_003_02', '01_Mamu_A1_003_w_09', '01_Mamu_A1_003g1', '01_Mamu_A1_004_SP_exp13277_ctg255_47_157bp', '01_Mamu_A1_004g', '01_Mamu_A1_006g', '01_Mamu_A1_007g1', '01_Mamu_A1_008g', '01_Mamu_A1_011g', '01_Mamu_A1_012_01', '01_Mamu_A1_016_01', '01_Mamu_A1_018_08', '01_Mamu_A1_023_A1_106g', '01_Mamu_A1_025_01', '01_Mamu_A1_026_01', '01_Mamu_A1_028_02', '01_Mamu_A1_028g1', '01_Mamu_A1_041_01', '01_Mamu_A1_055_01', '01_Mamu_A1_059_01', '01_Mamu_A1_059_nov_01', '01_Mamu_A1_066_01', '01_Mamu_A1_074g2', '01_Mamu_A1_109_01', '01_Mamu_A2_24_01', '01_Mamu_A7_01_w_02', '02_Mamu_A1_108g', '02_Mamu_A2_05_nov_05', '02_Mamu_A2_05g1', '02_Mamu_A2_05g2', '02_Mamu_A2_05g3', '02_Mamu_A2_05g5', '02_Mamu_A3_13_A4_03g', '02_Mamu_A3_13_A4_14g', '02_Mamu_A3_13g', '02_Mamu_A4_02_04', '02_Mamu_A4_14_07', '02_Mamu_A4_14_nov_01', '02_Mamu_A4_14_nov_03', '02_Mamu_A4_14_SP_exp13277_ctg134_142_157bp', '02_Mamu_A4_14g1', '02_Mamu_A4_14g2', '02_Mamu

                                            A008-A025  A008-A025  A016-A023  \
genotype                                                                      
01_Mamu_A1_001g2                                    0          0          0   
01_Mamu_A1_002_01                                   0          0          0   
01_Mamu_A1_002_w_02                                 0          0          0   
01_Mamu_A1_003_02                                   0          0          0   
01_Mamu_A1_003_w_09                                 0          0          0   
01_Mamu_A1_003g1                                    0          0          0   
01_Mamu_A1_004_SP_exp13277_ctg255_47_157bp          0          0          0   
01_Mamu_A1_004g                                     0          0          0   
01_Mamu_A1_006g                                     0          0          0   
01_Mamu_A1_007g1                                    0          0          0   
01_Mamu_A1_008g                                     

Step 3
* If you did not encounter any errors previously, or the output did not direct you to stop, then proceed with the analysis.
* You can either click this cell, select the Cell menu above, and then select 'Run All below', or click the cells after this one and individually click 'Run' for each one
* The analysis within this python notebook will do the following:
  * parse out genotype ID's for the MHC-A, and MHC-B data, 
  * use a pre-trained model for a Machine Learning Classifier to predict the Haplotype for each sample for MHC-A and MHC-B
  * add the predicted values back into the pivot table (or create a new pivot table)
  * output the resulting pivot table
* Note that for beta 1.x builds, only MHC-A and MHC-B haplotypes will be predicted.  All other haplotypes will be passed to the researcher for analysis.  A researcher should also verify the output pivot table from the Classifier.